In [2]:
import pandas as pd
from rdkit import Chem
import matplotlib.pyplot as plt
from rdkit.Chem import AllChem
import tqdm

In [3]:
df = pd.read_csv('./41586_2019_917_MOESM4_ESM.csv').iloc[:-4]
df.head()

,ZINC ID,Global Rank∗,Clustered Rank†,Energy,TC to knowns‡,Cosest neighbor among known DRD4 binders,Top-pick or not,Just from energy window,Energy window,Tested or not,Binder or not,D4 Ki(nM),D2 Ki(nM),D3 Ki(nM),cAMP EC50(nM),Inhibition (%) at 10uM,SMILES,Vendor ID,Charge from docked poses
0,ZINC000191583186,1,1,-75.50,0.30,ZINC000028347504,0,1,-75,1,1,1390.0,3860,1730,NT||,82.48,Cc1ccc(C[C@@H](CO)N[C@@H](C)CCc2ccccc2[N+](=O)...,Z1804039468,1.0
1,ZINC000159533726,2,2,-73.67,0.33,ZINC000103232405,0,1,-75,1,0,NaN,NaN,NaN,NT,2.68,C[C@H](C(=O)Nc1cc([N+](=O)[O-])ccc1Cl)N(C)C[C@...,Z1514931360,1.0
2,ZINC000151228439,3,4,-73.47,0.34,ZINC000053274848,0,1,-75,1,0,NaN,NaN,NaN,NT,17.5,C[C@@H](NC[C@](C)(O)c1ccccc1)c1cn(-c2ccccc2)nn1,Z1419817479,1.0
3,ZINC000291023493,5,5,-72.95,0.31,ZINC000028363497,0,1,-75,1,0,NaN,NaN,NaN,NT,-10.33,C[C@H](Nc1cc(-n2cccn2)nc(N)n1)[C@H](c1ccccc1)N...,Z2179794811,2.0
4,ZINC000593577820,7,7,-72.50,0.35,ZINC000036216606,0,1,-75,1,0,NaN,NaN,NaN,NT,11.48,COC(=O)C[C@H]1CSCCN1Cc1cn(-c2cccc(C)c2)nc1C,Z2480456501,1.0


In [4]:
dock_energy = df['Energy'].astype(float).values
inhibition = df['Inhibition (%) at 10uM'].astype(float).values
smiles = df['SMILES']
names = df['ZINC ID']

In [10]:
mols = [Chem.MolFromSmiles(i) for i in smiles]

In [7]:
with Chem.SDWriter('./mols.sdf') as writer:
    for m, n in tqdm.tqdm_notebook(zip(mols, names), total=len(mols)):
        mH = Chem.AddHs(m)
        AllChem.EmbedMolecule(mH)
        mH.SetProp('_Name', n)
        writer.write(mH)

/var/folders/jh/02165y2n7kq2y5ychxtzcjm40000gn/T/ipykernel_55970/209296595.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for m, n in tqdm.tqdm_notebook(zip(mols, names), total=len(mols)):


  0%|          | 0/549 [00:00<?, ?it/s]

# At this stage, dock all the compounds
i.e. run docking with 
```
smina - protein_5WIU.pdbqt -l mols.sdf --autobox_ligand ligand_5WIU.pdb -o docked.sdf
```